In [117]:
import os
import shutil
import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import glob

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

import mysql.connector as mysql
import pymysql
from sqlalchemy import create_engine
host=os.getenv('MYSQL_SERVER')
database=os.getenv('MYSQL_DATABASE')
user=os.getenv('MYSQL_USER')
password=os.getenv('MYSQL_PASSWORD')

import internal_functions as fn

IMPUTATIONVALUES = "'A','B','C','D','G','H','J','K','L','N','P','R','Y','Z',''"

### Remove and Create the /sql directory

In [126]:
directory = "sql"

# Remove the directory if it exists
if os.path.exists(directory):
    shutil.rmtree(directory)

# Create the directory
os.mkdir(directory)

### DROP & CREATE database

In [127]:
### Connect to database and create cursor
db, cursor = fn.create_dbconnection()
dropDBQuery = "DROP DATABASE IF EXISTS tim7020;"
createDBQuery = "CREATE DATABASE tim7020;"
fn.execute_dbquery(query=dropDBQuery, db=db, cursor=cursor)
fn.execute_dbquery(query=createDBQuery, db=db, cursor=cursor)

### Create new table names using OpenAI API using the column description

In [128]:
### 
### Process the IPEDS data tables and create new column names using OpenAI API
###

### Read the dictionary files for the database
dbDictionary = pd.read_excel('@dictionary.xlsx', sheet_name='Tables21')
dbValues = pd.read_excel('@dictionary.xlsx', sheet_name='valuesets21')

### Create a list of tuples with the table names and the data files
tableNames = list(zip(dbDictionary.TableName, dbDictionary.TIM7020TableName))
### Get a list of the tables that have already been processed
processedTables = fn.return_list_of_processed_tables()
###
### For each table, generate the new variable names using OpenAI API
### Write the new variable names to a ".new." dictionary file, 
###  so we're not having to make repeated calls to the API
###
for oldTableName, newTableName in tableNames:
    if oldTableName.upper() not in processedTables:
        ### Record the table we are working on
        print(f'oldTableName: {oldTableName}, newTableName: {newTableName}')
        ### Read table's dictionary file
        tableDictionary = pd.read_excel(f'dictionary/{oldTableName}.xlsx', sheet_name=1)
        # Create list of column descriptions to be used as input to the API
        varibleTitles = tableDictionary.iloc[:,6].tolist()
        # Print the number of variables in the list of column names
        print(f'varibleTitles (length): {len(varibleTitles)}')
        # Group the column names into groups of 30 to make the API calls
        groups = [varibleTitles[i:i+20] for i in range(0, len(varibleTitles), 20)]
        varibleTitlesNew = []
        for group in groups:
            varibleTitlesNew.extend(fn.create_name_from_description(group))
            time.sleep(5)
        # Print the number of variables in the list of new column names
        print(f'varibleTitlesNew (length): {len(varibleTitlesNew)}')
        # Append the new column names to the table's dictionary file
        tableDictionary['varname_new'] = varibleTitlesNew
        # Write the new dictionary file to disk
        tableDictionary.to_excel(f'dictionary/{oldTableName.lower()}.new.xlsx', sheet_name='varlist', index=False)


### Build CREATE and ALTER TABLE queries for the database

In [129]:
### Connect to database and create cursor
db, cursor = fn.create_dbconnection()
###
### Read the dictionary file for the database
###
dbDictionary = pd.read_excel('@dictionary.xlsx', sheet_name='Tables21')
dbRefValues = pd.read_excel('@dictionary.xlsx', sheet_name='valuesets21')
loopCount = 0
### Create a list of tuples with the table names and the data files
tableNames = list(zip(dbDictionary.TableName, dbDictionary.TIM7020TableName))

### 
### For each table, read the dictionary file and build the CREATE TABLE/ALTER TABLE queries 
### to build the IPEDS Postsecondary database using the IPEDS dictionary and data files
### The novel opportunity here is to use the dictory file to determine the data type of the columns
###   create new human readable column names and added the column description as column comments 
###
for oldTableName, newTableName in tableNames:
    ### Record the table we are working on
    print(f'oldTableName: {oldTableName}, newTableName: {newTableName}')
    ### Read table's dictionary file
    tableDictionary = pd.read_excel(f'dictionary/{oldTableName}.new.xlsx', sheet_name='varlist')
    # Create list of column descriptions less the first row (IPEDS ID)
    variableNames = tableDictionary['varname_new'].tolist()
    ### Read first 100 rows of the data file to verify data types
    tableData = pd.read_csv(f'data/{oldTableName}.csv', nrows=100, encoding="utf-8", na_values=['.', '. ', ' '])
    ### Create string for the query
    createTableQuery = None
    alterTableQuery = f'ALTER TABLE {newTableName} \n'
    ###
    ### Loop through the rows of the table dictionary file
    ###
    for index, row in tableDictionary.iterrows():
        ### Setup the column creation variables
        addColumn = None
        addImputed = None
        addCheck = None
        dataType = None
        ### Skip the first row (inst_id), we'll create the table with this common column
        if index == 0:
            createTableQuery = f'CREATE TABLE {newTableName} ({row.varname_new} INTEGER);'
            continue

        ### If there is an imputation variable, add the column to the table
        if (isinstance(row.imputationvar, str) == True) and (row.imputationvar != 'None' and row.imputationvar[0].upper() == 'X'):
            ###
            ### Create the ADD COLUMN line for the imputed flag column to be added to the ALTER TABLE query
            ### 
            addImputed = f'  ADD COLUMN {row.varname_new}_imp ENUM({IMPUTATIONVALUES}) COMMENT "{row.varname}|{row.varTitle}",'
        ###
        ### Determine the data type to use for the field based on the metadata
        ###
        dtype = ''
        if tableData[row.varname.upper()].dtype == 'float':
            dtype = 'float'
        dataType = fn.dtype_by_format(row.DataType, dtype, row.Fieldwidth)
        ###
        ### Create the ADD COLUMN line to be added to the ALTER TABLE query
        ###    
        addColumn = f'    ADD COLUMN {row.varname_new} {dataType} COMMENT "{row.varname}|{row.varTitle}",'
        ###
        ### Build the ALTER TABLE query for the current column
        ###
        addToQuery = ''
        if addImputed != None:
            addToQuery = addImputed + '\n'
        if addColumn != None:
            addToQuery += addColumn + '\n'
        alterTableQuery += addToQuery

        ### Continue for the next row in the table dictionary file
        continue
    ###
    ### Remove the last comma and space, add a semicolon
    ###
    alterTableQuery = alterTableQuery[:-2] + ';'

    ###
    ### Write the DROP, CREATE, ALTER TABLE queries to a SQL script file
    ###
    dropTableQuery = f'DROP TABLE IF EXISTS {newTableName};'
    fn.write_sql_file(f'{newTableName}.1.drop table', dropTableQuery)
    fn.write_sql_file(f'{newTableName}.2.create table', createTableQuery)
    fn.write_sql_file(f'{newTableName}.3.add columns', alterTableQuery)

    ###
    ### Execute the DROP, CREATE, and ALTER the table SQL queries
    ###
    fn.execute_dbquery(query=dropTableQuery, db=db, cursor=cursor)
    fn.execute_dbquery(query=createTableQuery, db=db, cursor=cursor)
    fn.execute_dbquery(query=alterTableQuery, db=db, cursor=cursor)

oldTableName: HD2021, newTableName: institution
oldTableName: FLAGS2021, newTableName: institution_ic_response_flags
oldTableName: IC2021, newTableName: institution_ic_offerings
oldTableName: IC2021_AY, newTableName: institution_ic_academic_charges
oldTableName: IC2021_PY, newTableName: institution_ic_vocational_charges
oldTableName: IC2021_CAMPUSES, newTableName: institution_ic_campuses
oldTableName: ADM2021, newTableName: institution_adm_considerations
oldTableName: EFFY2021, newTableName: institution_fy_enroll_headcount
oldTableName: EFIA2021, newTableName: institution_fy_enroll_contact
oldTableName: EFFY2021_DIST, newTableName: institution_fy_enroll_dist
oldTableName: EF2021A, newTableName: institution_f21_enroll_race_gender
oldTableName: EF2021B, newTableName: institution_f21_enroll_age_gender
oldTableName: EF2021C, newTableName: institution_f21_y1_residence
oldTableName: EF2021D, newTableName: institution_f21_y1_retention
oldTableName: EF2021A_DIST, newTableName: institution_f21_

### Upload CSV files into database tables

In [130]:
### Connect to database and create cursor
db, cursor = fn.create_dbconnection()
engine = fn.create_dbengine()
###
### Read the dictionary file for the databasea
###
dbDictionary = pd.read_excel('@dictionary.xlsx', sheet_name='Tables21')
dbRefValues = pd.read_excel('@dictionary.xlsx', sheet_name='valuesets21')
loopCount = 0
### Create a list of tuples with the table names and the data files
tableNames = list(zip(dbDictionary.TableName, dbDictionary.TIM7020TableName))

### 
### For each table, read the dictionary file and build the CREATE TABLE/ALTER TABLE queries 
### to build the IPEDS Postsecondary database using the IPEDS dictionary and data files
### The novel opportunity here is to use the dictory file to determine the data type of the columns
###   create new human readable column names and added the column description as column comments 
###
for oldTableName, newTableName in tableNames:
    ### Record the table we are working on
    print(f'oldTableName: {oldTableName}, newTableName: {newTableName}')
    ### Read table's dictionary file
    tableDictionary = pd.read_excel(f'dictionary/{oldTableName}.new.xlsx', sheet_name='varlist')
    ### Create a dictionary of old column names and new column names
    columnNames = dict(zip(tableDictionary['varname'], tableDictionary['varname_new']))
    impColumnNames = tableDictionary[tableDictionary.imputationvar.str.len() > 2][['imputationvar', 'varname_new']]
    impColumnNames.varname_new = impColumnNames.varname_new + '_imp'
    impColumnNames = dict(zip(impColumnNames['imputationvar'], impColumnNames['varname_new']))
    tableData = pd.read_csv(f'data/{oldTableName.lower()}.csv', encoding='latin1', na_values=['.', '. ', ' '])
    ### Rename the columns  
    tableData = tableData.rename(columns=columnNames)
    tableData = tableData.rename(columns=impColumnNames)
    ### Write data to database
    tableData.to_sql(name=newTableName, con=engine, if_exists='append', index=False)

oldTableName: HD2021, newTableName: institution
oldTableName: FLAGS2021, newTableName: institution_ic_response_flags
oldTableName: IC2021, newTableName: institution_ic_offerings
oldTableName: IC2021_AY, newTableName: institution_ic_academic_charges
oldTableName: IC2021_PY, newTableName: institution_ic_vocational_charges
oldTableName: IC2021_CAMPUSES, newTableName: institution_ic_campuses
oldTableName: ADM2021, newTableName: institution_adm_considerations
oldTableName: EFFY2021, newTableName: institution_fy_enroll_headcount
oldTableName: EFIA2021, newTableName: institution_fy_enroll_contact
oldTableName: EFFY2021_DIST, newTableName: institution_fy_enroll_dist
oldTableName: EF2021A, newTableName: institution_f21_enroll_race_gender
oldTableName: EF2021B, newTableName: institution_f21_enroll_age_gender
oldTableName: EF2021C, newTableName: institution_f21_y1_residence
oldTableName: EF2021D, newTableName: institution_f21_y1_retention
oldTableName: EF2021A_DIST, newTableName: institution_f21_

### Set PK constraints

In [131]:
### Connect to database and create cursor
db, cursor = fn.create_dbconnection()
engine = fn.create_dbengine()
###
### Read the dictionary file for the databasea
###
dbDictionary = pd.read_excel('@dictionary.xlsx', sheet_name='Tables21')
dbRefValues = pd.read_excel('@dictionary.xlsx', sheet_name='valuesets21')
loopCount = 0
### Create a list of tuples with the table names and the data files
tableNames = list(zip(dbDictionary.TableName, dbDictionary.TIM7020TableName))

### 
### For each table, read the dictionary file and build the CREATE TABLE/ALTER TABLE queries 
### to build the IPEDS Postsecondary database using the IPEDS dictionary and data files
### The novel opportunity here is to use the dictory file to determine the data type of the columns
###   create new human readable column names and added the column description as column comments 
###
for oldTableName, newTableName in tableNames:
    ### Record the table we are working on
    print(f'oldTableName: {oldTableName}, newTableName: {newTableName}')
    ### Read table's dictionary file
    tableDictionary = pd.read_excel(f'dictionary/{oldTableName}.new.xlsx', sheet_name='varlist')
    ### Create a list of columns with a PK designation
    pkColumns = list(tableDictionary[tableDictionary['imputationvar'] == 'PK']['varname_new'])
    addPKQuery = f'ALTER TABLE {newTableName}\n    ADD PRIMARY KEY ({", ".join(pkColumns)});'

    ###
    ### Write the ALTER TABLE query to a SQL script file
    ###
    fn.write_sql_file(f'{newTableName}.4.add PK', addPKQuery)

    ###
    ### Execute the ALTER TABLE SQL query
    ###
    fn.execute_dbquery(query=addPKQuery, db=db, cursor=cursor)


oldTableName: HD2021, newTableName: institution
oldTableName: FLAGS2021, newTableName: institution_ic_response_flags
oldTableName: IC2021, newTableName: institution_ic_offerings
oldTableName: IC2021_AY, newTableName: institution_ic_academic_charges
oldTableName: IC2021_PY, newTableName: institution_ic_vocational_charges
oldTableName: IC2021_CAMPUSES, newTableName: institution_ic_campuses
oldTableName: ADM2021, newTableName: institution_adm_considerations
oldTableName: EFFY2021, newTableName: institution_fy_enroll_headcount
oldTableName: EFIA2021, newTableName: institution_fy_enroll_contact
oldTableName: EFFY2021_DIST, newTableName: institution_fy_enroll_dist
oldTableName: EF2021A, newTableName: institution_f21_enroll_race_gender
oldTableName: EF2021B, newTableName: institution_f21_enroll_age_gender
oldTableName: EF2021C, newTableName: institution_f21_y1_residence
oldTableName: EF2021D, newTableName: institution_f21_y1_retention
oldTableName: EF2021A_DIST, newTableName: institution_f21_

### Create reference tables for discrete table columns

In [132]:
### Connect to database and create cursor
db, cursor = fn.create_dbconnection()
engine = fn.create_dbengine()
###
### Read the dictionary file for the databasea
###
dbDictionary = pd.read_excel('@dictionary.xlsx', sheet_name='Tables21')
dbRefValues = pd.read_excel('@dictionary.xlsx', sheet_name='valuesets21')
loopCount = 0
### Create a list of tuples with the table names and the data files
tableNames = list(zip(dbDictionary.TableName, dbDictionary.TIM7020TableName))

### 
### For each table, read the dictionary file and build the CREATE TABLE/ALTER TABLE queries 
### to build the IPEDS Postsecondary database using the IPEDS dictionary and data files
### The novel opportunity here is to use the dictory file to determine the data type of the columns
###   create new human readable column names and added the column description as column comments 
###
for oldTableName, newTableName in tableNames:
    ### Record the table we are working on
    print(f'oldTableName: {oldTableName}, newTableName: {newTableName}')
    ### Skip the IC2021_CAMPUSES table, as it uses reference tables from other tables
    if oldTableName in ['IC2021_CAMPUSES']: #,'HD2021',  'FLAGS2021', 'IC2021', 'IC2021_AY', 'IC2021_PY', 'ADM2021', 'EFFY2021', 'EFIA2021', 'EFFY2021_DIST', 'EF2021A', 'EF2021B', 'EF2021C', 'EF2021D', 'EF2021A_DIST']:
        continue

    ### Read the reference values for the table
    tableDictionary = pd.read_excel(f'dictionary/{oldTableName.lower()}.new.xlsx', sheet_name='varlist')
    tableData = pd.read_csv(f'data/{oldTableName.lower()}.csv', nrows=100, encoding="utf-8", na_values=['.', '. ', ' '])
    ### Filter tableDictionary to only the rows format is equal to Disc, select only varname and varname_new columns
    tableDictionary = tableDictionary[tableDictionary.format == 'Disc'].sort_values(by=['varname'])

    allRefValues = dbRefValues[dbRefValues.TableName == oldTableName]
    allRefValues = allRefValues.sort_values(by=['varName', 'valueOrder'])

    for _, row in tableDictionary.iterrows():
        # Print which colomn are we working on
        # print(f'{row.DataType}, {row.Fieldwidth}, {row.varname}, {row.varname_new}')

        # filter allRefValues to only the rows where varName is equal to the current row varname
        refValues = allRefValues[allRefValues.varName == row.varname]
        ## if refValues is empty, continue to the next row
        if refValues.empty:
            # print(f'oldTableName: {oldTableName}, newTableName: {newTableName}')
            print(f'No refValues for {row.varname}, {row.varname_new}')
            continue
        
        ### Create DROP TABLE query
        dropRefTableQuery = f'DROP TABLE IF EXISTS institution_xref_{row.varname_new};'
        ### CREATE CREATE TABLE query
        # Determine the data type of the column
        dtype = ''
        if tableData[row.varname.upper()].dtype == 'float':
            dtype = 'float'
        dataType = fn.dtype_by_format(row.DataType, dtype, row.Fieldwidth)
        # Build the CREATE TABLE query text
        createRefTableQuery = f'''
            CREATE TABLE institution_xref_{row.varname_new} (
                Codevalue {dataType},
                valueLabel VARCHAR(255),
                valueOrder MEDIUMINT UNSIGNED,
                PRIMARY KEY (Codevalue)
            );'''
        ###
        ### Write the DROP, CREATE, ALTER TABLE queries to a SQL script file
        ###
        fn.write_sql_file(f'institution_xref_{row.varname_new}.1.drop table', dropRefTableQuery)
        fn.write_sql_file(f'institution_xref_{row.varname_new}.2.create table', createRefTableQuery)
        ###
        ### Execute the DROP, CREATE and INSERT queries
        ###
        fn.execute_dbquery(query=dropRefTableQuery, db=db, cursor=cursor)
        fn.execute_dbquery(query=createRefTableQuery, db=db, cursor=cursor)
        refValues = refValues[['Codevalue', 'valueLabel', 'valueOrder']]
        refValues.to_sql(name=f'institution_xref_{row.varname_new}', con=engine, if_exists='append', index=False)   


oldTableName: HD2021, newTableName: institution
oldTableName: FLAGS2021, newTableName: institution_ic_response_flags
oldTableName: IC2021, newTableName: institution_ic_offerings
oldTableName: IC2021_AY, newTableName: institution_ic_academic_charges
oldTableName: IC2021_PY, newTableName: institution_ic_vocational_charges
oldTableName: IC2021_CAMPUSES, newTableName: institution_ic_campuses
oldTableName: ADM2021, newTableName: institution_adm_considerations
oldTableName: EFFY2021, newTableName: institution_fy_enroll_headcount
oldTableName: EFIA2021, newTableName: institution_fy_enroll_contact
oldTableName: EFFY2021_DIST, newTableName: institution_fy_enroll_dist
oldTableName: EF2021A, newTableName: institution_f21_enroll_race_gender
oldTableName: EF2021B, newTableName: institution_f21_enroll_age_gender
oldTableName: EF2021C, newTableName: institution_f21_y1_residence
oldTableName: EF2021D, newTableName: institution_f21_y1_retention
oldTableName: EF2021A_DIST, newTableName: institution_f21_

### Create FK contraints

In [133]:
### Connect to database and create cursor
db, cursor = fn.create_dbconnection()
engine = fn.create_dbengine()
###
### Read the dictionary file for the databasea
###
dbDictionary = pd.read_excel('@dictionary.xlsx', sheet_name='Tables21')
dbRefValues = pd.read_excel('@dictionary.xlsx', sheet_name='valuesets21')
###
### Create a list of child tables to apply foreign key constraints
###
dbDictionary = dbDictionary[dbDictionary.TIM7020TableName != 'institution']

loopCount = 0

### Create a list of tuples with the table names and the data files
tableNames = list(zip(dbDictionary.TableName, dbDictionary.TIM7020TableName))
### 
### For each table, read the dictionary file and build the ALTER TABLE queries 
###     to create the foreign keys constraints to the 'institution' parent table or
###     the 'institution_xref_{column name}' reference table
###
for oldTableName, newTableName in tableNames:
    ### Record the table we are working on
    print(f'oldTableName: {oldTableName}, newTableName: {newTableName}')
    tableDictionary = pd.read_excel(f'dictionary/{oldTableName.lower()}.new.xlsx', sheet_name='varlist')
    tableDictionary = tableDictionary[tableDictionary.format == 'Disc']

    if oldTableName in ['IC2021_CAMPUSES']:
        continue

    alterTableQuery = f'''
        ALTER TABLE {newTableName} 
            ADD FOREIGN KEY (inst_id) REFERENCES institution(inst_id)'''
    
    for _, row in tableDictionary.iterrows():
        ### Filter dbRefValues for the current variable
        refValues = dbRefValues[dbRefValues.varName == row.varname]
        ### If the column data type is not numeric or the number of unique values is less than 10, 
        ### Then create CHECK constraint 
        ### Else FK constraint

        if row['DataType'] == 'A' or len(refValues) > 10:
            alterTableQuery += f''', 
            ADD FOREIGN KEY ({row.varname_new}) REFERENCES institution_xref_{row.varname_new}(Codevalue)'''
        else:
            alterTableQuery += f''', 
            ADD CHECK ({row.varname_new} IN ({", ".join(refValues.Codevalue.to_list())}))'''

    alterTableQuery += ';'
    ###
    ### Write the ALTER Table FK query to a file
    ###
    fn.write_sql_file(f'{newTableName}.5.add FK-Check', alterTableQuery)
    ###
    ### Execute the ALTER Table FK query
    ###
    fn.execute_dbquery(query=alterTableQuery, db=db, cursor=cursor)


oldTableName: FLAGS2021, newTableName: institution_ic_response_flags
oldTableName: IC2021, newTableName: institution_ic_offerings
oldTableName: IC2021_AY, newTableName: institution_ic_academic_charges
oldTableName: IC2021_PY, newTableName: institution_ic_vocational_charges
oldTableName: IC2021_CAMPUSES, newTableName: institution_ic_campuses
oldTableName: ADM2021, newTableName: institution_adm_considerations
oldTableName: EFFY2021, newTableName: institution_fy_enroll_headcount
oldTableName: EFIA2021, newTableName: institution_fy_enroll_contact
oldTableName: EFFY2021_DIST, newTableName: institution_fy_enroll_dist
oldTableName: EF2021A, newTableName: institution_f21_enroll_race_gender
oldTableName: EF2021B, newTableName: institution_f21_enroll_age_gender
oldTableName: EF2021C, newTableName: institution_f21_y1_residence
oldTableName: EF2021D, newTableName: institution_f21_y1_retention
oldTableName: EF2021A_DIST, newTableName: institution_f21_dist
oldTableName: C2021_A, newTableName: instit